In [1]:
import numpy as np

In [2]:
B = 0.7
M = np.array([[0.0, 0.0, 0.0],
              [0.5, 0.0, 0.0],
              [0.5, 1.0, 1.0]])

In [5]:
def page_rank(M, tax, n_iter=10, E=None):
    """
    Parameters
    ---------
        M : array of shape=(n, n), connectivity matrix.
        tax : amound of tax, for example 0.8
        E : epsilon, min difference between previous and current rank vector to proceed the next iteration
    """
    r = np.ones(shape=M.shape[0]) / M.shape[0]
    r_new = np.zeros_like(r)
    e = np.ones_like(r) / M.shape[0]
    
    for i in range(n_iter):
        r_new = tax * M@r + (1-tax) * e
        print(f'After iter {i+1}: {r}')
        
        # check convergance
        if E is not None:
            if len(np.nonzero(r - r_new < E)[0]) == len(r):
                print(f'Converged after {i+1} iterations')
                break
        
        r = r_new
        
    return r

In [6]:
r_res = page_rank(M, tax=1.0, n_iter=10)

After iter 1: [ 0.33333333  0.33333333  0.33333333]
After iter 2: [ 0.          0.16666667  0.83333333]
After iter 3: [ 0.  0.  1.]
After iter 4: [ 0.  0.  1.]
After iter 5: [ 0.  0.  1.]
After iter 6: [ 0.  0.  1.]
After iter 7: [ 0.  0.  1.]
After iter 8: [ 0.  0.  1.]
After iter 9: [ 0.  0.  1.]
After iter 10: [ 0.  0.  1.]


In [7]:
a, b, c = r_res[0], r_res[1], r_res[2]

In [8]:
print(b, 0.575*a + 0.15*c)
print(0.95*a, 0.9*c + 0.05*b)
print(0.85*c, b + 0.575*a)
print(a, c + 0.15*b)

0.0 0.15
0.0 0.9
0.85 0.0
0.0 1.0


In [9]:
def ts_page_rank(M, tax, S=None, n_iter=1, E=None):
    """
    Parameters
    ---------
        M : array of shape=(n, n), connectivity matrix.
        tax : amound of tax, 
            that is the probability of random walking. 1-tax is the probability that it jumps to some node in S.
        S : array of shape=(n,), with ones indicating nodes in topic set.
        n_iter : int, number of iterations
        E : epsilon, min difference between previous and current rank vector to proceed the next iteration
    """
    r = np.ones(shape=M.shape[0]) / M.shape[0]
    r_new = np.zeros_like(r)
    
    if S is None:
        S = np.ones(shape=M.shape[0])
    S = S / np.sum(S)
    
    for i in range(n_iter):
        r_new = tax * M@r + (1-tax) * S
        print(f'After iter {i+1}: {r_new}')
        
        # check convergance
        if E is not None:
            if len(np.nonzero(r - r_new < E)[0]) == len(r):
                print(f'Converged after {i+1} iterations')
                break
        
        r = r_new
        
    return r

In [10]:
r_res = ts_page_rank(M, tax=0.85, n_iter=10, E=0.001)

After iter 1: [ 0.05        0.19166667  0.75833333]
After iter 2: [ 0.05     0.07125  0.87875]
After iter 3: [ 0.05     0.07125  0.87875]
Converged after 3 iterations


In [11]:
def hubs_authorities(M, tax, n_iter=1, E=None):
    n = M.shape[0]
    a = np.ones(shape=n)
    h = np.ones(shape=n)
    e = np.ones(shape=n) / n
    # initialize
    a = a / np.sqrt(n)
    h = h / np.sqrt(n)
    A = tax * M + (1-tax) * e
    
    for i in range(n_iter):
        # calc new values
        h_new = A@a
        a_new = A.T@h
        
        # normalize
        h_new = h_new / np.sum(h_new)
        a_new = a_new / np.sum(a_new)
         
        print(f'After iter {i+1}: hub scores: {h_new}, auth scores: {a_new}')
        
        # check convergance
        if E is not None:
            if len(np.nonzero((h - h_new)**2 < E)[0]) == len(h) and len(np.nonzero((a - a_new)**2 < E)[0]) == len(a):
                print(f'Converged after {i+1} iterations')
                break
        
        h = h_new
        a = a_new
        
    return h, a
    

In [12]:
tax = 1.0
M = np.array([[1.0, 1.0, 1.0],
              [1.0, 0.0, 1.0],
              [0.0, 1.0, 0.0]])

hubs_authorities(M, tax=tax, n_iter=3, E=0.01)

After iter 1: hub scores: [ 0.5         0.33333333  0.16666667], auth scores: [ 0.33333333  0.33333333  0.33333333]
After iter 2: hub scores: [ 0.5         0.33333333  0.16666667], auth scores: [ 0.35714286  0.28571429  0.35714286]
Converged after 2 iterations


(array([ 0.5       ,  0.33333333,  0.16666667]),
 array([ 0.33333333,  0.33333333,  0.33333333]))

Pearson